In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32


In [2]:
# define data generators for train,validation and test sets
train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [3]:


train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/lung_cancer',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/lung_cancer',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/lung_cancer',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 192 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [4]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator,validation_data=val_generator,epochs=5)

model.save("lung.h5","label.txt")


Epoch 1/5
6/6 [==============================] - 33s 4s/step - loss: 0.9124 - accuracy: 0.6875 - val_loss: 0.3349 - val_accuracy: 0.8750
Epoch 2/5
6/6 [==============================] - 23s 4s/step - loss: 0.0554 - accuracy: 1.0000 - val_loss: 0.1387 - val_accuracy: 0.9375
Epoch 3/5
6/6 [==============================] - 23s 4s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 2.1678 - val_accuracy: 0.7917
Epoch 4/5
6/6 [==============================] - 23s 4s/step - loss: 1.1369e-04 - accuracy: 1.0000 - val_loss: 3.0254 - val_accuracy: 0.7917
Epoch 5/5
6/6 [==============================] - 25s 4s/step - loss: 3.8545e-06 - accuracy: 1.0000 - val_loss: 3.0018 - val_accuracy: 0.7917


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
#load the saved model
model=load_model('/content/lung.h5')
#load the preprocess the test imaage
test_image_path='/content/drive/MyDrive/Dataset/lung_cancer/Malignant cases/000000 (6).png'
img = image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)#Add batch dimension
img_array /=225. #Normalize the pixel values
#Make prediction
prediction=model.predict(img_array)
#Print the prediction
if prediction < 0.5:
  print("Prediction:no lung cancer (Probability:",prediction[0][0],")")
else:
    print("Prediction:lung cancer present  (Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 109ms/step
Prediction:lung cancer present  (Probability: 1.0 )
